# FINAL CODE STEPS FOR SEARCH ENGINE PROJECT

## Step 1: Reading the data from the database

### Part 1

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import sqlite3

conn = sqlite3.connect("Data/eng_subtitles_database.db")

In [ ]:
import pandas as pd

query = 'SELECT * FROM zipfiles'

df = pd.read_sql_query(query, conn)

conn.close()

df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [ ]:
df.shape

(82498, 3)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


### Part 2

In [ ]:
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

In [ ]:
import zipfile
import io
def decomp_decode(data):
    with zipfile.ZipFile(io.BytesIO(data)) as zip_file:
        # Extract the first file in the ZIP archive
        file_list = zip_file.namelist()
        first_file = file_list[0]
        decompressed_data = zip_file.read(first_file)
    return decompressed_data.decode('latin-1')

In [ ]:
df['content'] = df['content'].progress_apply(lambda x : decomp_decode(x))

100%|██████████████████████████████████████████████████████████████████████████| 82498/82498 [00:26<00:00, 3125.53it/s]


In [ ]:
df['content'].head()

0    1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...
1    1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther...
2    1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'...
3    1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...
4    ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...
Name: content, dtype: object

## Step 2: Data cleaning

In [ ]:
import re

def clean_data(data): # data is the entire text file entry in the dataframe

    # removing timestamps
    data = re.sub("\d{2}:\d{2}:\d{2},\d{3}\s-->\s\d{2}:\d{2}:\d{2},\d{3}"," ",  data)

    # removing index no. of dialogues
    data = re.sub(r'\n?\d+\r', "", data)

    # removing escape sequences like \n \r
    data = re.sub('\r|\n', "", data)

    # removing <i> and </i>
    data = re.sub('<i>|</i>', "", data)
    # removing links
    data = re.sub("(?:www\.)osdb\.link\/[\w\d]+|www\.OpenSubtitles\.org|osdb\.link\/ext|api\.OpenSubtitles\.org|OpenSubtitles\.com", " ",data)

    # Converting to lower case
    data = data.lower()

    # return
    return data

In [ ]:
df['content'] = df['content'].progress_apply(lambda x: clean_data(x))

100%|███████████████████████████████████████████████████████████████████████████| 82498/82498 [04:54<00:00, 280.38it/s]


In [ ]:
df['num'] = df['num'].apply(lambda x : str(x)) #converting 'num' id to string as chromaDB ids need to be in string format

## Step 3: Data chunking - semantic chunking

### Part 1: Creating the chunking function

In [ ]:
#!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model_name = 'paraphrase-MiniLM-L3-v2' #all-MiniLM-L6-v2
model = SentenceTransformer(model_name, device='cuda')

def semantic_chunking(document, similarity_threshold=0.9):

    # Tokenize the document into sentences
    sentences = document.split('.')

    # Initialize variables for semantic chunks
    chunks = []
    current_chunk = sentences[0]

    # Generate embeddings for the sentences
    sentence_embeddings = model.encode(sentences)

    # Iterate over the sentences and group semantically similar sentences into chunks
    for i in range(1, len(sentences)):
        # Calculate cosine similarity between the current sentence and the previous sentence
        similarity_score = np.dot(sentence_embeddings[i], sentence_embeddings[i-1]) / (np.linalg.norm(sentence_embeddings[i]) * np.linalg.norm(sentence_embeddings[i-1]))

        # If similarity score is above the threshold, add the sentence to the current chunk
        if similarity_score >= similarity_threshold:
            current_chunk += '.' + sentences[i]
        else:
            # If similarity score is below the threshold, start a new chunk
            chunks.append(current_chunk)
            current_chunk = sentences[i]

    # Add the last chunk
    chunks.append(current_chunk)

    return chunks

### Part 2: Running the function in batches

In [ ]:
# temporary dataframes to split the data into two parts

temp_1 = pd.DataFrame()
temp_2 = pd.DataFrame()

In [ ]:
temp_1['num']=df['num'][:30000]

In [ ]:
temp_2['num']=df['num'][30000:]

In [ ]:
# 1st section using joblib for parallel processing on the first part of the data

from joblib import Parallel, delayed
import time
start = time.time()
temp_1['chunks'] = Parallel(n_jobs=-1)(delayed(semantic_chunking)(item) for item in df['content'].values[:30000])
end=time.time()
print(f"Total time in seconds = {end-start}")

Total time in seconds = 4418.998434782028


In [ ]:
#saving to json file

temp_1.to_json("database.json") #saving data to json file to restrart the kernel and save RAM

In [ ]:
# 2nd section - using joblib for parallel processing on the second part of the data
from joblib import Parallel, delayed
import time
start = time.time()
temp_2['chunks'] = Parallel(n_jobs=-1)(delayed(semantic_chunking)(item) for item in df['content'].values[30000:])
end=time.time()
print(f"Total time in seconds = {end-start}")

Total time in seconds = 8644.886986494064


In [ ]:
temp_2.to_json("database_p2.json") #saving data to json file to restrart the kernel and save RAM

In [ ]:
# restarting the kernel
# interacting with each part of the json file

import json

json_file_path = "database.json" #database_p2.json
with open(json_file_path, 'r') as f:
  data = json.load(f)

df = pd.DataFrame(data)

## Step 4: Generating text embeddings

### Part 1: Preparing the indexes

In [ ]:
#creating index for the data

def indexer(item):
    index=[]
    temp=int(df[df['num']==item].index[0])
    for j in range(len(df['chunks'].iloc[temp])):
        index.append(item+"-"+str(j))# since id needs to be unique adding the j index with a hyphen to create a unique id
    return index

In [ ]:
df['num_list'] = df['num'].apply(lambda x : indexer(x)) #indexing the embeddings

### Part 2: Creating the text embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = 'paraphrase-MiniLM-L3-v2' #all-MiniLM-L6-v2
model = SentenceTransformer(model_name, device='cuda')

In [ ]:
def embedding_gen(data):
  return model.encode(data).tolist()

In [ ]:
df['embeddings'] = Parallel(n_jobs=-1)(delayed(embedding_gen)(item) for item in df['chunks'].values)

## Step 5:  Storing data in ChromaDB

### Setting up chromaDB

In [ ]:
import chromadb
client = chromadb.PersistentClient(path="E://search_engine_db")
collection = client.get_or_create_collection(name="search_engine", metadata={"hnsw:space": "cosine"})
collection_2 = client.get_or_create_collection(name="search_engine_FileName", metadata={"hnsw:space": "cosine"})

### Creating function to add filenames of our subtitles

In [ ]:
# Ran this part already before splitting data into 2 temporary dataframes
def add_func_v1():
    for i in range(df.shape[0]): #setting the range as total no. of rows in dataframe
        collection_2.add(
            documents=[df['name'].iloc[i]], # adding each filename
            embeddings=[[1,2,34,45]], # adding a random data, as we don't need it when retrieving file_name
            ids=[df['num'].iloc[i]] # entering unique 'num' id
        )

### Creating function to add the chunks, embeddings and unique identifiers for our subtitle files

In [ ]:
def add_func_v2():
    for i in range(df.shape[0]): #setting the range as total no. of rows in dataframe
        collection.add(
            documents=df['chunks'].iloc[i], # adding each chunk
            embeddings=df['embeddings'].iloc[i], # adding the corresponding chunk embedding
            ids=df['num_list'].iloc[i] #entering the unique 'num' id
        )

In [ ]:
%time add_func_v1()

In [ ]:
%time add_func_v2()

## Step 6: Creating the streamlit app

In [ ]:
# streamlit code
# command to run : streamlit run app.py

### Part 1: Importing the necessary libraries

In [ ]:
import re
import chromadb
from sentence_transformers import  SentenceTransformer
import streamlit as st

### Part 2: Initializing chromaDB


In [ ]:
client = chromadb.PersistentClient(path="E://search_engine_db") #_test_db
collection = client.get_collection(name="search_engine") #test_collection
collection_name = client.get_collection(name="search_engine_FileName")
model_name="paraphrase-MiniLM-L3-V2"
model = SentenceTransformer(model_name, device="cuda")

### Part 3: Cleaning steps for the user query

In [ ]:
def clean_data(data): # data is the query text

    # removing timestamps
    data = re.sub("\d{2}:\d{2}:\d{2},\d{3}\s-->\s\d{2}:\d{2}:\d{2},\d{3}"," ",  data)

    # removing index no. of dialogues
    data = re.sub(r'\n?\d+\r', "", data)

    # removing escape sequences like \n \r
    data = re.sub('\r|\n', "", data)

    # removing <i> and </i>
    data = re.sub('<i>|</i>', "", data)
    # removing links
    data = re.sub("(?:www\.)osdb\.link\/[\w\d]+|www\.OpenSubtitles\.org|osdb\.link\/ext|api\.OpenSubtitles\.org|OpenSubtitles\.com", " ",data)

    # Converting to lower case
    data = data.lower()

    # return
    return data

### Part 4: Creating a function to extract the subtitle_id

In [ ]:
def extract_id(id_list):
    new_id_list=[]
    for item in id_list:
        match = re.match(r'^(\d+)', item)
        if match:
            extracted_number = match.group(1)
            new_id_list.append(extracted_number)
    return new_id_list

### Part 5: Creating the web application

In [ ]:
st.header("Movie Subtitle Search Engine")
search_query=st.text_input("Enter a dialogue to search....")
if st.button("Search")==True:

    st.subheader("Relevant Subtitle Files")
    search_query=clean_data(search_query)
    query_embed = model.encode(search_query).tolist()

    search_results=collection.query(query_embeddings=query_embed, n_results=10)
    id_list = search_results['ids'][0]

    id_list = extract_id(id_list)
    print(id_list)
    for id in id_list:
        file_name = collection_name.get(ids=f"{id}")["documents"][0]
        st.markdown(f"[{file_name}](https://www.opensubtitles.org/en/subtitles/{id})")
